In [157]:
from datascience import Table # Including Table functions from datascience package into the running instance
import os# including os package into running instance
from datascience.predicates import are# Import 'are' from 'predicates' in 'datascience' module for comparison
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt  # To visualize
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
%matplotlib inline
from sklearn.linear_model import LinearRegression

#---------------------------------------------------------------------------------------------------------------------

#LOAD DATA

#---------------------------------------------------------------------------------------------------------------------

#Loading data from CSV file URL address 
InputFilepath='https://raw.githubusercontent.com/babloomarkose/DataInPython/master/ShelfLifePrediction'
ProductTable=Table().read_table(InputFilepath+'/Dataset.csv')

#Print the Data Table
print("The Data Table")
#ProductTable.show(5) # Show only the first 5 records

#---------------------------------------------------------------------------------------------------------------------

#EXPLORATORY DATA ANALYSIS

#---------------------------------------------------------------------------------------------------------------------

print("Number of Records : " ,  ProductTable.num_rows)
print("Number of Variables : ",ProductTable.num_columns)
print("Variables : ",ProductTable.labels)

#Find Number of Missing Values in each column
print("Missing Values in Study Number : ",ProductTable.num_rows-sum(not np.isnan(x) for x in ProductTable['Study Number']))
print("Missing Values in Sample ID : ",(ProductTable['Sample ID'] == 'nan').sum())

print("Missing Values in Process Type : ",(ProductTable['Process Type'] == 'nan').sum())
print("Missing Values in Product Type : ",(ProductTable['Product Type'] == 'nan').sum())
print("Missing Values in Processing Agent Stability Index : ",ProductTable.num_rows-sum(not np.isnan(x) for x in ProductTable['Processing Agent Stability Index']))
print("Missing Values in Sample Age (Weeks) : ",ProductTable.num_rows-sum(not np.isnan(x) for x in ProductTable['Sample Age (Weeks)']))
print("Missing Values in Difference From Fresh : ",ProductTable.num_rows-sum(not np.isnan(x) for x in ProductTable['Difference From Fresh']))

print("Missing Values in Base Ingredient : ",(ProductTable['Base Ingredient'] == 'nan').sum())
print("Missing Values in Moisture (%) : ",ProductTable.num_rows-sum(not np.isnan(x) for x in ProductTable['Moisture (%)']))
print("Missing Values in Packaging Stabilizer Added : ",(ProductTable['Packaging Stabilizer Added'] == 'nan').sum())
print("Missing Values in Storage Conditions : ",(ProductTable['Storage Conditions'] == 'nan').sum())
print("Missing Values in Residual Oxygen (%) : ",ProductTable.num_rows-sum(not np.isnan(x) for x in ProductTable['Residual Oxygen (%)']))
print("Missing Values in Hexanal (ppm) : ",ProductTable.num_rows-sum(not np.isnan(x) for x in ProductTable['Hexanal (ppm)']))
print("Missing Values in Preservative Added : ",(ProductTable['Preservative Added'] == 'nan').sum())
print("Missing Values in Transparent Window in Package : ",(ProductTable['Transparent Window in Package'] == 'nan').sum())


'''ProductTable.pivot_hist('Product Type','Base Ingredient',label='Product Type to Base Ingredient')
ProductTable.pivot_hist('Product Type','Process Type',label='Product Type to Process Type')
ProductTable.pivot_hist('Product Type','Storage Conditions',label='Product Type to Storage Conditions')
ProductTable.pivot_hist('Product Type','Packaging Stabilizer Added',label='Product Type to Packaging Stabilizer Added')
plt.show()'''

#Count The number of Unique Values in string variables
print('Product Type:',Counter(ProductTable['Product Type'] ))
print('Base Ingredient: ',Counter(ProductTable['Base Ingredient'] ))
print('Process Type: ',Counter(ProductTable['Process Type'] ))
print('Storage Conditions: ',Counter(ProductTable['Storage Conditions'] ))
print('Packaging Stabilizer Added: ',Counter(ProductTable['Packaging Stabilizer Added'] ))
print('Preservative Added: ',Counter(ProductTable['Preservative Added'] ))
print('Transparent Window in Package: ',Counter(ProductTable['Transparent Window in Package'] ))
print('Processing Agent Stability Index: ',Counter(ProductTable['Processing Agent Stability Index'] ))

#Remove Transparent Window in Package from further consideration as non-null values  belong to only one category 
ProductTable=ProductTable.drop('Transparent Window in Package')
#Also Residual Oxygen, Moisture (%) and Hexanal (ppm) are charecteristics of aged samples. 
#Hence is of little use for prediction of shelf life. hence droping those columns also from further analysis
ProductTable=ProductTable.drop('Moisture (%)')
ProductTable=ProductTable.drop('Residual Oxygen (%)')
ProductTable=ProductTable.drop('Hexanal (ppm)')
#Also drop colomn Study Number/Sample ID as this is not affecting shelf life in anyway
ProductTable=ProductTable.drop('Study Number')
ProductTable=ProductTable.drop('Sample ID')

#Drop Duplicate Entries is there are any
pandasDF=ProductTable.to_df()
pandasDF.drop_duplicates(keep='first',inplace=True)#inplace=True modify original record
ProductTable=Table.from_df(pandasDF)

# I am considering 'Difference From Fresh' as my dependent variable. And once the prediction is done and we have an 
#equation, substitute 20 for it to find the shelf life of any given product
ProductTable=ProductTable.move_column('Difference From Fresh',0)
ProductTable=ProductTable.move_column('Sample Age (Weeks)',1)
ProductTable=ProductTable.move_column('Processing Agent Stability Index',2)
ProductTable=ProductTable.move_column('Process Type',3)

#Transforming variable Process Type
# 3 Categories in Process Type,Process Type:  Counter({'C': 534, 'A': 167, 'B': 48})
#so add 2 dummy variable ProcessTypeC,ProcessTypeA,ProcessTypeB
#Add two new colomns and initialise to 0
#--------------------------------------------------------------------------------------------------------------------
#ONE HOT ENCODING
#--------------------------------------------------------------------------------------------------------------------
ProductTable.append_column('ProcessTypeC', False)
ProductTable.append_column('ProcessTypeB', False)
ProductTable.append_column('ProcessTypeA', False)
ProductTable['ProcessTypeA']=ProductTable.apply(lambda x: True if x=='A' else False,'Process Type')
ProductTable['ProcessTypeC']=ProductTable.apply(lambda x: True if x=='C' else False,'Process Type')
ProductTable['ProcessTypeB']=ProductTable.apply(lambda x: True if x=='B' else False,'Process Type')

ProductTable=ProductTable.drop('Process Type')

#Transforming variable Product Type
ProductTable.append_column('ProductTypeA',False)
ProductTable.append_column('ProductTypeB',False)
ProductTable.append_column('ProductTypeC',False)
ProductTable.append_column('ProductTypeD',False)
ProductTable.append_column('ProductTypeE',False)
ProductTable.append_column('ProductTypeF',False)
ProductTable.append_column('ProductTypeG',False)
ProductTable.append_column('ProductTypeH',False)
ProductTable.append_column('ProductTypeI',False)
ProductTable['ProductTypeA']=ProductTable.apply(lambda x: True if x=='A' else False,'Product Type')
ProductTable['ProductTypeB']=ProductTable.apply(lambda x: True if x=='B' else False,'Product Type')
ProductTable['ProductTypeC']=ProductTable.apply(lambda x: True if x=='C' else False,'Product Type')
ProductTable['ProductTypeD']=ProductTable.apply(lambda x: True if x=='D' else False,'Product Type')
ProductTable['ProductTypeE']=ProductTable.apply(lambda x: True if x=='E' else False,'Product Type')
ProductTable['ProductTypeF']=ProductTable.apply(lambda x: True if x=='F' else False,'Product Type')
ProductTable['ProductTypeG']=ProductTable.apply(lambda x: True if x=='G' else False,'Product Type')
ProductTable['ProductTypeH']=ProductTable.apply(lambda x: True if x=='H' else False,'Product Type')
ProductTable['ProductTypeI']=ProductTable.apply(lambda x: True if x=='I' else False,'Product Type')

ProductTable=ProductTable.drop('Product Type')

#Transforming variable Product Type
#When base ingredient is not given all these fields will be 0
ProductTable.append_column('BaseIngredientA',False)
ProductTable.append_column('BaseIngredientB',False)
ProductTable.append_column('BaseIngredientC',False)
ProductTable.append_column('BaseIngredientD',False)
ProductTable.append_column('BaseIngredientE',False)
ProductTable.append_column('BaseIngredientF',False)
ProductTable['BaseIngredientA']=ProductTable.apply(lambda x: True if x=='A' else False,'Base Ingredient')
ProductTable['BaseIngredientB']=ProductTable.apply(lambda x: True if x=='B' else False,'Base Ingredient')
ProductTable['BaseIngredientC']=ProductTable.apply(lambda x: True if x=='C' else False,'Base Ingredient')
ProductTable['BaseIngredientD']=ProductTable.apply(lambda x: True if x=='D' else False,'Base Ingredient')
ProductTable['BaseIngredientE']=ProductTable.apply(lambda x: True if x=='E' else False,'Base Ingredient')
ProductTable['BaseIngredientF']=ProductTable.apply(lambda x: True if x=='F' else False,'Base Ingredient')

ProductTable=ProductTable.drop('Base Ingredient')

#Transforming variable Packaging Stabilizer Added
#When information on Packaging Stabilizer Added is not given, these two fields will be 0
ProductTable.append_column('PackagingStabilizerAdded',False)
ProductTable['PackagingStabilizerAdded']=ProductTable.apply(lambda x: True if x=='Y' else False,'Packaging Stabilizer Added')
ProductTable.append_column('PackagingStabilizerNotAdded',False)
ProductTable['PackagingStabilizerNotAdded']=ProductTable.apply(lambda x: True if x=='N' else False,'Packaging Stabilizer Added')
ProductTable=ProductTable.drop('Packaging Stabilizer Added')

#Transforming variable Storage Conditions
#When information on Storage Conditions is not given, these three fields will be 0
ProductTable.append_column('WarmClimate',False)
ProductTable.append_column('ColdClimate',False)
ProductTable.append_column('HighTemperatureandHumidity',False)
ProductTable['WarmClimate']=ProductTable.apply(lambda x: True if x=='Warm Climate' else False,'Storage Conditions')
ProductTable['ColdClimate']=ProductTable.apply(lambda x: True if x=='Cold Climate' else False,'Storage Conditions')
ProductTable['HighTemperatureandHumidity']=ProductTable.apply(lambda x: True if x=='High Temperature and Humidity' else False,'Storage Conditions')
ProductTable=ProductTable.drop('Storage Conditions')

#Transforming variable Preservative Added
#When information on Preservative Added is not given, these two fields will be 0
ProductTable.append_column('PreservativeAdded',False)
ProductTable.append_column('PreservativeNotAdded',False)
ProductTable['PreservativeAdded']=ProductTable.apply(lambda x: True if x=='Y' else False,'Preservative Added')
ProductTable['PreservativeNotAdded']=ProductTable.apply(lambda x: True if x=='N' else False,'Preservative Added')
ProductTable=ProductTable.drop('Preservative Added')

#Add new colomn Fresh based on 'Difference From Fresh' True for value less than or equal to 20 else False
ProductTable.append_column('Fresh',True)
ProductTable['Fresh']=ProductTable.apply(lambda x: True if x<=20 else False,'Difference From Fresh')

#Analyze relation between Dependent variable Difference From Fresh and Sample Age (Weeks)
'''ProductTable.scatter('Sample Age (Weeks)','Difference From Fresh')
ProductTable.scatter('Processing Agent Stability Index','Difference From Fresh')
plt.show()'''

#---------------------------------------------------------------------------------------------------------------------

#LINEAR REGRESSION

#---------------------------------------------------------------------------------------------------------------------
pandasDF=ProductTable.to_df()

X = pandasDF[['ProductTypeA','ProductTypeB','ProductTypeC','ProductTypeD','ProductTypeE','ProductTypeF','ProductTypeG','ProductTypeH','ProductTypeI',
                  'BaseIngredientA','BaseIngredientB','BaseIngredientC','BaseIngredientD','BaseIngredientE','BaseIngredientF',
                  'ProcessTypeA','ProcessTypeB','ProcessTypeC',
                  'Difference From Fresh','WarmClimate','ColdClimate','HighTemperatureandHumidity',
                  'PackagingStabilizerAdded','PackagingStabilizerNotAdded',
                  'Processing Agent Stability Index','PreservativeAdded','PreservativeNotAdded']] 

Y = pandasDF['Sample Age (Weeks)']
# Create linear regression object.
mlr= LinearRegression(normalize=True)

# Fit linear regression.
mlr.fit( X, Y)                  

# Get the slope and intercept of the line best fit.
print(mlr.intercept_)
print(mlr.coef_)

# Make predictions using the testing set
Y_PRED_Combined = mlr.predict(X)

# The mean squared error
print(" Combined data Mean squared error: %.2f"% mean_squared_error(Y, Y_PRED_Combined))

# Explained variance score: 1 is perfect prediction
print('For Combined Dataset Variance score: %.2f' % r2_score(Y, Y_PRED_Combined))

print('Data Set prediction r2:', mlr.score(X, Y))

ProductTable.append_column('Difference From Fresh20',19)
pandasDF_withColumnDifference_From_Fresh20=ProductTable.to_df()
X_New_Prediction=pandasDF_withColumnDifference_From_Fresh20[['ProductTypeA','ProductTypeB','ProductTypeC','ProductTypeD','ProductTypeE','ProductTypeF','ProductTypeG','ProductTypeH','ProductTypeI',
                  'BaseIngredientA','BaseIngredientB','BaseIngredientC','BaseIngredientD','BaseIngredientE','BaseIngredientF',
                  'ProcessTypeA','ProcessTypeB','ProcessTypeC',
                  'Difference From Fresh20','WarmClimate','ColdClimate','HighTemperatureandHumidity',
                  'PackagingStabilizerAdded','PackagingStabilizerNotAdded',
                  'Processing Agent Stability Index','PreservativeAdded','PreservativeNotAdded']]
ProductTable.append_column('ShelfLife',mlr.predict(X_New_Prediction))
ProductTable.show()

# Plot outputs
'''plt.scatter( X_Test,Y_PRED_Test,  color='black')
plt.plot( X_Test,Y_PRED_Test,  color='blue')
plt.scatter( X_Train,Y_PRED_Train,  color='black')
plt.plot( X_Train,Y_PRED_Train,  color='red')
plt.show()'''


The Data Table
Number of Records :  749
Number of Variables :  15
Variables :  ('Study Number', 'Sample ID', 'Product Type', 'Base Ingredient', 'Process Type', 'Sample Age (Weeks)', 'Difference From Fresh', 'Storage Conditions', 'Packaging Stabilizer Added', 'Transparent Window in Package', 'Processing Agent Stability Index', 'Preservative Added', 'Moisture (%)', 'Residual Oxygen (%)', 'Hexanal (ppm)')
Missing Values in Study Number :  0
Missing Values in Sample ID :  0
Missing Values in Process Type :  0
Missing Values in Product Type :  0
Missing Values in Processing Agent Stability Index :  0
Missing Values in Sample Age (Weeks) :  0
Missing Values in Difference From Fresh :  0
Missing Values in Base Ingredient :  109
Missing Values in Moisture (%) :  236
Missing Values in Packaging Stabilizer Added :  282
Missing Values in Storage Conditions :  294
Missing Values in Residual Oxygen (%) :  329
Missing Values in Hexanal (ppm) :  460
Missing Values in Preservative Added :  480
Missing

Difference From Fresh,Sample Age (Weeks),Processing Agent Stability Index,ProcessTypeC,ProcessTypeB,ProcessTypeA,ProductTypeA,ProductTypeB,ProductTypeC,ProductTypeD,ProductTypeE,ProductTypeF,ProductTypeG,ProductTypeH,ProductTypeI,BaseIngredientA,BaseIngredientB,BaseIngredientC,BaseIngredientD,BaseIngredientE,BaseIngredientF,PackagingStabilizerAdded,PackagingStabilizerNotAdded,WarmClimate,ColdClimate,HighTemperatureandHumidity,PreservativeAdded,PreservativeNotAdded,Fresh,Difference From Fresh20,ShelfLife
0,0,16.2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,19,21.375
0,10,16.2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,19,21.375
0,14,16.2,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,19,21.375
3.3,16,16,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,19,21.25
2.7,18,16,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,19,21.25
0,0,16,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,19,13.5
34,5,16,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,19,13.5
22,11,16,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,19,13.5
44,13,16,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,19,13.5
50,15,16,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,19,13.5


"plt.scatter( X_Test,Y_PRED_Test,  color='black')\nplt.plot( X_Test,Y_PRED_Test,  color='blue')\nplt.scatter( X_Train,Y_PRED_Train,  color='black')\nplt.plot( X_Train,Y_PRED_Train,  color='red')\nplt.show()"